# Mixer correction
In this simple tutorial we will demonstrate the ability to compensate for output mixer non-idealities and observe the changes using an oscilloscope.

Mixer non-idealities can lead to unwanted spurs on the output (LO/RF/IF feedthrough and other spurious products) and they can be compensated by applying adjustments to the I/Q outputs: phase offset, gain ratio and DC offset. This solution applies to both baseband QCM/QRM products using external mixers as well as QCM-RF and QRM-RF products.

The tutorial is designed with a baseband Pulsar QRM in mind, but also can easily be used with all QBLOX baseband products: Pulsar QRM, Pulsar QCM, Cluster QRM, Cluster QCM. We will adjust all the parameters listed above and observe the changes to the I/Q outputs directly on an oscilloscope.

The tutorial can also work with QCM-RF and QRM-RF products, but the effects can only be observed indirectly by connecting a signal analyzer to the mixed output.

Requirements: 
- Oscilloscope
- Install ipywidgets: "pip install ipywidgets"
- Changes to notebook extensions: "jupyter nbextension enable --py widgetsnbextension"

In [1]:
#Set up environment
import pprint
import os
import json
import ipywidgets as widgets
from ipywidgets import interact

#Add Pulsar QRM interface
from pulsar_qrm.pulsar_qrm import pulsar_qrm

#Close any existing connections to any pulsar_qrm
pulsar_qrm.close_all()

#Connect to the Pulsar QRM at default IP address.
pulsar = pulsar_qrm("qrm", "192.168.0.2")

#Reset the instrument for good measure.
pulsar.reset()
print("Status:")
print(pulsar.get_system_status())

Status:
{'status': 'OKAY', 'flags': []}


Setup
-----

The easiest way to view the influence of the mixer correction is to mix the NCO sin and cos with I and Q values of 1 (fullscale). The instrument output would be simple sinusoids with a 90deg phase offset and identical amplitude.

We use sequencer 0 to set I and Q values of 1 (fullscle) using DC offset and we mix those with the NCO signals.

In [2]:
#Program sequence we will not use
wave_and_prog_dict = {"waveforms": {}, "weights": {}, "acquisitions": {}, "program": "stop"}
with open("sequence.json", 'w', encoding='utf-8') as file:
    json.dump(wave_and_prog_dict, file, indent=4)
    file.close()
pulsar.sequencer0_waveforms_and_program(os.path.join(os.getcwd(), "sequence.json"))

#Program fullscale DC offset on I & Q, turn on NCO and enable modulation
pulsar.sequencer0_offset_awg_path0(1.0)
pulsar.sequencer0_offset_awg_path1(1.0)
pulsar.sequencer0_nco_freq(10e6)
pulsar.sequencer0_mod_en_awg(True)

Control sliders
-----

Create control sliders for the parameters described in the introduction. Each time the value of a parameter is updated, the sequencer is automatically stopped from the embedded firmware for safety reasons and has to be manually restarted.

The sliders cover the valid parameter range. If the code below is modified to input invalid values, the Pulsar firmware will not program the values.

Please connect the I/Q outputs to an oscilloscope and set to trigger continuously on the I channel at 0V. Execute the code below, move the sliders and observe the result on the oscilloscope.

In [3]:
def set_offset_I(offset_I):
    pulsar.out0_offset(offset_I)
    pulsar.arm_sequencer(0)
    pulsar.start_sequencer(0)
    
def set_offset_Q(offset_Q):
    pulsar.out1_offset(offset_Q)
    pulsar.arm_sequencer(0)
    pulsar.start_sequencer(0)
    
def set_gain_ratio(gain_ratio):
    pulsar.sequencer0_mixer_corr_gain_ratio(gain_ratio)
    pulsar.arm_sequencer(0)
    pulsar.start_sequencer(0)
    
def set_phase_offset(phase_offset):
    pulsar.sequencer0_mixer_corr_phase_offset_degree(phase_offset)
    pulsar.arm_sequencer(0)
    pulsar.start_sequencer(0)
    
interact(set_offset_I, offset_I=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, start=0.0))
interact(set_offset_Q, offset_Q=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, start=0.0))
interact(set_gain_ratio, gain_ratio=widgets.FloatSlider(min=0.5, max=2.0, step=0.1, start=0.5))
interact(set_phase_offset, phase_offset=widgets.FloatSlider(min=-45.0, max=45.0, step=1.0, start=0.0))

interactive(children=(FloatSlider(value=0.0, description='offset_I', max=1.0, min=-1.0, step=0.01), Output()),…

interactive(children=(FloatSlider(value=0.0, description='offset_Q', max=1.0, min=-1.0, step=0.01), Output()),…

interactive(children=(FloatSlider(value=0.5, description='gain_ratio', max=2.0, min=0.5), Output()), _dom_clas…

interactive(children=(FloatSlider(value=0.0, description='phase_offset', max=45.0, min=-45.0, step=1.0), Outpu…

<function __main__.set_phase_offset(phase_offset)>

Stop
----

Finally, let's stop the sequencer and close the instrument connection.

In [4]:
#Stop sequencers
pulsar.stop_sequencer()

#Print sequencer status (should now say it is stopped).
print("Status:")
print(pulsar.get_sequencer_state(0))

#Print an overview of the instrument parameters.
print("Snapshot:")
pulsar.print_readable_snapshot(update=True)

#Close the instrument connection.
pulsar.close()

Status:
{'status': 'STOPPED', 'flags': ['FORCED STOP']}
Snapshot:
qrm:
	parameter                                  value
--------------------------------------------------------------------------------
IDN                                         :	{'manufacturer': 'Qblox', 'mode...
in0_gain                                    :	-6 (dB)
in1_gain                                    :	-6 (dB)
out0_offset                                 :	0 (V)
out1_offset                                 :	0 (V)
reference_source                            :	internal 
scope_acq_avg_mode_en_path0                 :	False 
scope_acq_avg_mode_en_path1                 :	False 
scope_acq_sequencer_select                  :	0 
scope_acq_trigger_level_path0               :	0 
scope_acq_trigger_level_path1               :	0 
scope_acq_trigger_mode_path0                :	sequencer 
scope_acq_trigger_mode_path1                :	sequencer 
sequencer0_channel_map_path0_out0_en        :	True 
sequencer0_channel_map_path1_o